In [2]:
#-----------------------------------staring with the whole data set by Study Type-----------------------------------#
#importing all the essential library # from this line!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

df=pd.read_csv('All_Studies.csv.', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
df.head()

,nct_id,study_type,phase,start_month_year,primary_completion_month_year,has_dmc,overall_status,brief_title,why_stopped,enrollment_type,...,lead_Sponsor,Agency_Class_Lead,lead_or_collaborator,subjects_affected_binary,subjects_at_risk_binary,responsible_party_type,enrollment_bins,minimum_age_bins,maximum_age_bins,stopped
0,NCT04163796,Interventional,Not_Provided,1-Aug-18,30-Oct-21,TRUE,Recruiting,UPnRIDE Power Standing Wheelchair for SCI,not stopped/ not stated,Anticipated,...,"Bronx Veterans Medical Research Foundation, Inc",Other,lead,FALSE,FALSE,Principal Investigator,VLOW,Adults,Older Adults,not stopped/ not stated
1,NCT03264222,Interventional,Not_Provided,1-May-17,18-Sep,FALSE,Unknown status,Safety of a New Security Device in Patients Wi...,not stopped/ not stated,Anticipated,...,Deutsches Herzzentrum Muenchen,Other,collaborator$$lead,FALSE,FALSE,Sponsor,VLOW,Adults,no_criteria,not stopped/ not stated
2,NCT02118870,Interventional,Phase 4,10-Jun-14,17-Sep,TRUE,Completed,Short-term Dual Anti Platelet Therapy in Patie...,not stopped/ not stated,Actual,...,Diagram B.V.,Other,lead,FALSE,FALSE,Sponsor,MED,Adults,no_criteria,not stopped/ not stated
3,NCT04175977,Interventional,Not_Provided,1-Jan-20,22-Aug,TRUE,Enrolling by invitation,The Hospice Advanced Dementia Symptom Manageme...,not stopped/ not stated,Anticipated,...,NYU Langone Health,Other,collaborator$$lead,FALSE,FALSE,Sponsor,HIGH,Adults,no_criteria,not stopped/ not stated
4,NCT00682370,Interventional,Phase 1,7-Oct,8-Jul,FALSE,Completed,Effects of Heme Arginate in Healthy Male Subjects,not stopped/ not stated,Actual,...,Daniel Doberer,Other,lead,FALSE,FALSE,Sponsor-Investigator,VLOW,Adults,Adults,not stopped/ not stated


In [3]:
#Dropping the uninformative feature
del df['nct_id']
df.dtypes

study_type                       object
phase                            object
start_month_year                 object
primary_completion_month_year    object
has_dmc                          object
overall_status                   object
brief_title                      object
why_stopped                      object
enrollment_type                  object
study_first_submitted_date       object
number_of_facilities_bins        object
number_of_arms_bins              object
number_of_groups                 object
country_name                     object
condition_name                   object
actual_duration_bins             object
were_results_reported            object
has_single_facility              object
allocation                       object
masking                          object
intervention_model               object
primary_purpose                  object
observational_model              object
gender                           object
healthy_volunteers               object


In [4]:
df.describe(include='all')

,study_type,phase,start_month_year,primary_completion_month_year,has_dmc,overall_status,brief_title,why_stopped,enrollment_type,study_first_submitted_date,...,lead_Sponsor,Agency_Class_Lead,lead_or_collaborator,subjects_affected_binary,subjects_at_risk_binary,responsible_party_type,enrollment_bins,minimum_age_bins,maximum_age_bins,stopped
count,269402,269402,269402,269402,269402,269402,269402,269402,269402,269402,...,269402,269400,269402,269402,269402,269402,269402,269402,269402,269402
unique,4,9,6820,7153,3,13,268755,14932,3,6861,...,28036,4,2,3,3,5,7,5,6,2
top,Interventional,Not_Provided,not_provided,not_provided,FALSE,Completed,Sun Protection Factor Assay,not stopped/ not stated,Actual,9/13/2005,...,GlaxoSmithKline,Other,lead,FALSE,FALSE,Sponsor,VLOW,Adults,no_criteria,not stopped/ not stated
freq,210427,91642,2902,15047,139086,145241,10,249812,160895,1227,...,2396,192021,179406,242158,235252,138098,233799,174584,128756,249809


In [5]:
#Dropping all the null value
df.dropna(inplace = True)

In [6]:
#Getting all the categorical columns except the study type#
#///////////////////////////////////////__Analysing by STudy Type__///////////////////////////////////////#
categorical_columns = df.select_dtypes(exclude = 'number').drop('study_type', axis = 1).columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269400 entries, 0 to 269401
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   study_type                     269400 non-null  object
 1   phase                          269400 non-null  object
 2   start_month_year               269400 non-null  object
 3   primary_completion_month_year  269400 non-null  object
 4   has_dmc                        269400 non-null  object
 5   overall_status                 269400 non-null  object
 6   brief_title                    269400 non-null  object
 7   why_stopped                    269400 non-null  object
 8   enrollment_type                269400 non-null  object
 9   study_first_submitted_date     269400 non-null  object
 10  number_of_facilities_bins      269400 non-null  object
 11  number_of_arms_bins            269400 non-null  object
 12  number_of_groups               269400 non-nu

In [7]:
# Import the function
from scipy.stats import chi2_contingency
chi2_check = []
for i in categorical_columns:
    if chi2_contingency(pd.crosstab(df['study_type'], df[i]))[1] < 0.05:
        chi2_check.append('Reject Null Hypothesis')
    else:
        chi2_check.append('Fail to Reject Null Hypothesis')
res = pd.DataFrame(data = [categorical_columns, chi2_check] 
             ).T 
res.columns = ['Column', 'Hypothesis']
print(res)

                           Column                      Hypothesis
0                           phase          Reject Null Hypothesis
1                start_month_year          Reject Null Hypothesis
2   primary_completion_month_year          Reject Null Hypothesis
3                         has_dmc          Reject Null Hypothesis
4                  overall_status          Reject Null Hypothesis
5                     brief_title  Fail to Reject Null Hypothesis
6                     why_stopped  Fail to Reject Null Hypothesis
7                 enrollment_type          Reject Null Hypothesis
8      study_first_submitted_date          Reject Null Hypothesis
9       number_of_facilities_bins          Reject Null Hypothesis
10            number_of_arms_bins          Reject Null Hypothesis
11               number_of_groups          Reject Null Hypothesis
12                   country_name          Reject Null Hypothesis
13                 condition_name          Reject Null Hypothesis
14        

In [9]:
# Testing the function - 1 pair
from scipy.stats import chi2_contingency
#Testing the relationship
chi_res = chi2_contingency(pd.crosstab(df['study_type'], df['why_stopped']))
print('Chi2 Statistic: {}, p-value: {}'.format(chi_res[0], chi_res[1]))

Chi2 Statistic: 17748.807433013488, p-value: 1.0


In [64]:
property_dummies = pd.get_dummies(data = df[['study_type', 'primary_purpose']], columns = ['study_type'])
property_dummies

,primary_purpose,study_type_Interventional,study_type_Observational,study_type_Observational [Patient Registry]
0,Supportive Care,1,0,0
1,Prevention,1,0,0
2,Treatment,1,0,0
3,not_provided,0,1,0
4,Supportive Care,1,0,0
5,Treatment,1,0,0
6,Treatment,1,0,0
7,Treatment,1,0,0
8,Prevention,1,0,0
9,not_provided,0,1,0


In [65]:
pd.crosstab(property_dummies['primary_purpose'], property_dummies['study_type_Interventional'])

study_type_Interventional,0,1
primary_purpose,,
Basic Science,0,3
Diagnostic,0,1
Health Services Research,0,2
Prevention,0,5
Supportive Care,0,3
Treatment,0,15
not_provided,11,0


In [66]:
check = {}
for i in res[res['Hypothesis'] == 'Reject Null Hypothesis']['Column']:
    dummies = pd.get_dummies(df[i])
    bon_p_value = 0.05/df[i].nunique()
    for series in dummies:
        if chi2_contingency(pd.crosstab(df['study_type'], dummies[series]))[1] < bon_p_value:
            check['{}-{}'.format(i, series)] = 'Reject Null Hypothesis'
        else:
            check['{}-{}'.format(i, series)] = 'Fail to Reject Null Hypothesis'
res_chi_ph = pd.DataFrame(data = [check.keys(), check.values()]).T
res_chi_ph.columns = ['Pair', 'Hypothesis']
res_chi_ph
#res_chi_ph.to_csv("Capstone.ChiTest.csv")

,Pair,Hypothesis
0,phase-Not_Interventional,Reject Null Hypothesis
1,phase-Not_Provided,Fail to Reject Null Hypothesis
2,phase-Phase 1,Fail to Reject Null Hypothesis
3,phase-Phase 1/Phase 2,Fail to Reject Null Hypothesis
4,phase-Phase 2,Fail to Reject Null Hypothesis
...,...,...
149,maximum_age-70 Years,Fail to Reject Null Hypothesis
150,maximum_age-75 Years,Fail to Reject Null Hypothesis
151,maximum_age-85 Years,Fail to Reject Null Hypothesis
152,maximum_age-89 Years,Fail to Reject Null Hypothesis
